In [1]:
##############################
# Load the necessary libraries
##############################

import os
import math
import geopandas as gpd
import pandas as pd
import rtree
from shapely.geometry import Point, LineString
from shapely.ops import unary_union
from collections import defaultdict


###########################################
# Set Directory Paths and File Name Strings
###########################################

base_dir = r"C:\Users\natda\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

roads_filename = "street_network.shp"
nodes_filename = "nodes.shp"
merge_or_not_nodes_filename = "merge_or_not_nodes.shp"
junctions_filename = "junctions.shp"
crossings_filename = "crossings.shp"

roads_path = os.path.join(base_dir, roads_filename)
nodes_path = os.path.join(base_dir, nodes_filename)
merge_or_not_nodes_path = os.path.join(base_dir, merge_or_not_nodes_filename)
junctions_path = os.path.join(base_dir, junctions_filename)
crossings_path = os.path.join(base_dir, crossings_filename)

############################
# Load the Road Network Data
############################

roads_gdf = gpd.read_file(roads_path)
if 'unique_id' in roads_gdf.columns and roads_gdf.index.name != 'unique_id':
    roads_gdf = roads_gdf.set_index('unique_id', drop=False)

print("Loaded roads_gdf with CRS:", roads_gdf.crs)

Loaded roads_gdf with CRS: EPSG:6491


In [2]:
##############
# Create Nodes
##############

# Filter and create a dict for valid road segments
roads_dict = {}
for idx, row in roads_gdf.iterrows():
    qExclude = row.get('qExclude', 0)
    qNoAccess = row.get('qNoAccess', 0)
    geom = row.geometry

    # Only include valid segments in roads_dict
    if qExclude not in {1, 5} and qNoAccess != 1 and geom is not None:
        roads_dict[idx] = {
            'geometry': geom,
            'StOperNEU': row.get('StOperNEU', 0),
            'STREETNAME': row.get('STREETNAME', "")
        }

# Collect start and end points of valid road segments
endpoints_dict = defaultdict(list)
for sid, data in roads_dict.items():
    geom = data['geometry']

    # Handle geometry to get the very first and very last coordinates
    if geom.geom_type == "LineString":
        first_point = geom.coords[0]
        last_point = geom.coords[-1]
    elif geom.geom_type == "MultiLineString":
        first_point = geom.geoms[0].coords[0]     # Start of the first LineString
        last_point = geom.geoms[-1].coords[-1]    # End of the last LineString
    else:
        continue                                  # Skip unsupported geometries

    # Record the start and end points
    endpoints_dict[first_point].append(sid)
    endpoints_dict[last_point].append(sid)

# Identify potential intersections
nodes_list = []
node_id = 1

for pt, seg_ids in endpoints_dict.items():
    if len(seg_ids) >= 3:
        nodes_list.append({
            'NODE_ID': node_id,
            'INC_LINKS': seg_ids,
            'NUM_LINKS': len(seg_ids),
            'geometry': Point(pt)
        })
        node_id += 1

nodes_gdf = gpd.GeoDataFrame(nodes_list, crs=roads_gdf.crs)
nodes_gdf.to_file(nodes_path)
print(f"Saved {len(nodes_gdf)} nodes in total.")

Saved 13096 nodes in total.


In [3]:
##############################
# Identify Path-Crossing Nodes
##############################

# Identify links with "Off-Road Path"
off_road_links = set(roads_gdf.loc[roads_gdf['bike_type'] == "Off-Road Path", 'unique_id'])

# Check for path crossing nodes
def is_path_crossing(inc_links):
    # Count how many incident links are "Off-Road Path"
    off_road_count = sum(1 for link in inc_links if link in off_road_links)
    return off_road_count >= 2  # At least two off-road paths

# Apply the function and create a new column
nodes_gdf['PATH_XING'] = nodes_gdf['INC_LINKS'].apply(is_path_crossing)

# Count path-crossing and total nodes
num_path_crossing = nodes_gdf['PATH_XING'].sum()
total_nodes = len(nodes_gdf)

print(f"{num_path_crossing} out of {total_nodes} nodes are path-crossing nodes.")

# Overwrite the nodes_gdf shapefile
nodes_gdf.to_file(nodes_path)

559 out of 13096 nodes are path-crossing nodes.


In [4]:
###################################
# Build Spatial Index for Junctions
###################################
spatial_index = rtree.index.Index(
    (idx, node['geometry'].bounds, idx) for idx, node in enumerate(nodes_list)
)

def get_node(endpoint):
    """Find the exact node matching the endpoint."""
    candidates = [nodes_list[idx] for idx in spatial_index.intersection(Point(endpoint).bounds)]
    return next((n for n in candidates if n['geometry'].equals(Point(endpoint))), None)

def has_another_stoperneu_11(node, shared_segments):
    """Check if the node has another link on a divided highway."""
    return any(
        roads_dict[sid]['StOperNEU'] == 11 and sid not in shared_segments for sid in node['INC_LINKS']
    )

##########################################
# Flag Nodes for Inspection Using Indexing
##########################################

merge_or_not_nodes = []

for sid, road_data in roads_dict.items():
    if road_data['StOperNEU'] != 11:
        continue

    # Get endpoints
    geom = road_data['geometry']
    endpoints = (
        [geom.coords[0], geom.coords[-1]] if geom.geom_type == "LineString" else 
        [geom.geoms[0].coords[0], geom.geoms[-1].coords[-1]] if geom.geom_type == "MultiLineString" else
        None
    )
    if endpoints is None:
        continue

    # Find nodes
    node_a, node_b = get_node(endpoints[0]), get_node(endpoints[1])
    if not node_a or not node_b:
        continue

    # Check spatial and link conditions
    distance = node_a['geometry'].distance(node_b['geometry'])
    if not (14 <= distance <= 28):
        continue

    num_links_a, num_links_b = len(node_a['INC_LINKS']), len(node_b['INC_LINKS'])
    if not ((num_links_a == 3 and num_links_b == 4) or (num_links_a == 4 and num_links_b == 3)):
        continue

    # Check for individual divided road segments
    shared_segments = set(node_a['INC_LINKS']) & set(node_b['INC_LINKS'])
    if has_another_stoperneu_11(node_a, shared_segments) and has_another_stoperneu_11(node_b, shared_segments):
        # Add to inspection list
        merge_or_not_nodes.extend([
            {'NODE_ID': node_a['NODE_ID'], 'geometry': node_a['geometry']},
            {'NODE_ID': node_b['NODE_ID'], 'geometry': node_b['geometry']}
        ])

# Create GeoDataFrame and drop duplicates
merge_or_not_nodes_gdf = gpd.GeoDataFrame(merge_or_not_nodes, crs=roads_gdf.crs)
merge_or_not_nodes_gdf = merge_or_not_nodes_gdf.drop_duplicates(subset=['geometry'])

merge_or_not_nodes_gdf.to_file(merge_or_not_nodes_path)

print(f"There are {len(merge_or_not_nodes_gdf)} unique nodes to inspect (merge or not).")

There are 146 unique nodes to inspect (merge or not).


In [5]:
#############
# Merge Nodes
#############

def create_proper_junctions(nodes, roads, threshold):
    """
    Merge nodes within 'threshold' distance if they meet the conditions.
    Path-crossing nodes only merge with other path-crossing nodes,
    and non-path-crossing nodes only merge with other non-path-crossing nodes.
    """
    merged_nodes = []
    standalone_nodes = []
    processed = set()

    for node in nodes.itertuples():
        if node.Index in processed:
            continue

        # Find nearby nodes within the threshold distance
        close_nodes = nodes[nodes.geometry.distance(node.geometry) <= threshold]
        inc_links = set(node.INC_LINKS)
        constituent_nodes = {node.NODE_ID}  # Use a set to ensure unique node IDs

        # Track connecting links to exclude
        links_to_exclude = set()

        for cn in close_nodes.itertuples():
            # Skip already processed nodes
            if cn.Index in processed:
                continue

            # Ensure that only similar types of nodes are merged
            if cn.PATH_XING != node.PATH_XING:
                continue  # Do not merge different types of nodes (path vs. non-path)

            # Both nodes must have at least one segment with StOperNEU = 11
            has_highway_link_current = any(roads[sid]['StOperNEU'] == 11 for sid in node.INC_LINKS)
            has_highway_link_nearby = any(roads[sid]['StOperNEU'] == 11 for sid in cn.INC_LINKS)

            if not (has_highway_link_current and has_highway_link_nearby):
                continue  # Skip if either node does not meet the divided highway condition

            # Identify and exclude shared links or connecting links
            shared_links = set(node.INC_LINKS) & set(cn.INC_LINKS)

            # Detect connecting links by checking if their geometry connects the two nodes
            for link in shared_links:
                link_geom = roads[link]['geometry']
                if (
                    link_geom.geom_type == "LineString"
                    and (
                        link_geom.coords[0] == (node.geometry.x, node.geometry.y)
                        and link_geom.coords[-1] == (cn.geometry.x, cn.geometry.y)
                        or link_geom.coords[0] == (cn.geometry.x, cn.geometry.y)
                        and link_geom.coords[-1] == (node.geometry.x, node.geometry.y)
                    )
                ):
                    links_to_exclude.add(link)

            # Add the node to the merging group and exclude shared or connecting links
            constituent_nodes.add(cn.NODE_ID)
            inc_links.update(sid for sid in cn.INC_LINKS if sid not in links_to_exclude)

        # Finalize merging by ensuring unique links and nodes
        final_inc_links = list(set(inc_links) - links_to_exclude)
        final_constituent_nodes = list(constituent_nodes)

        # Check if merging will result in at least three unique links
        if len(final_constituent_nodes) > 1 and len(final_inc_links) >= 3:
            mg = unary_union([n.geometry for n in nodes.loc[nodes['NODE_ID'].isin(final_constituent_nodes)].itertuples()])
            merged_nodes.append({
                'INTER_ID': node.NODE_ID,
                'INC_LINKS': final_inc_links,  # Ensure unique links are stored
                'NUM_LINKS': len(final_inc_links),
                'geometry': mg.centroid,
                'WAS_MERGED': True,
                'CON_NODES': final_constituent_nodes  # Ensure unique node IDs are stored
            })
            processed.update(nodes.loc[nodes['NODE_ID'].isin(final_constituent_nodes)].index)
        else:
            # Keep the node as standalone if merging is invalid
            standalone_nodes.append({
                'INTER_ID': node.NODE_ID,
                'INC_LINKS': list(set(node.INC_LINKS)),  # Ensure unique incident links
                'NUM_LINKS': len(set(node.INC_LINKS)),
                'geometry': node.geometry,
                'WAS_MERGED': False,
                'CON_NODES': [node.NODE_ID]
            })
            processed.add(node.Index)

    mgdf = gpd.GeoDataFrame(merged_nodes, geometry='geometry', crs=nodes.crs)
    sgdf = gpd.GeoDataFrame(standalone_nodes, geometry='geometry', crs=nodes.crs)
    return pd.concat([mgdf, sgdf], ignore_index=True)


junctions_gdf = create_proper_junctions(nodes_gdf, roads_dict, threshold=28)
junctions_gdf.to_file(junctions_path)
print(f"Intersections saved: {len(junctions_gdf)} total.")

Intersections saved: 12352 total.


In [6]:
########################
# Create Legs Dictionary
########################

def calc_bearing(jxy, geom):
    """Compute bearing (0 to 360°, north=0) from intersection jxy outward."""
    if geom.geom_type == "LineString":
        coords = geom.coords
    elif geom.geom_type == "MultiLineString" and len(geom.geoms) > 0:
        coords = geom.geoms[0].coords
    else:
        return None

    if len(coords) < 2:
        return None

    sx, sy = coords[0]
    ex, ey = coords[-1]

    ds = (sx - jxy[0])**2 + (sy - jxy[1])**2
    de = (ex - jxy[0])**2 + (ey - jxy[1])**2

    if ds < de:
        dx, dy = ex - jxy[0], ey - jxy[1]
    else:
        dx, dy = sx - jxy[0], sy - jxy[1]

    angle_rad = math.atan2(dx, dy)
    deg = math.degrees(angle_rad)
    return deg + 360 if deg < 0 else deg

def point_from_bearing(jxy, bearing_deg, dist_m=6):
    """Compute a point dist_m away from jxy at bearing_deg (0=North, clockwise)."""
    theta = math.radians(bearing_deg)
    return (
        jxy[0] + dist_m * math.sin(theta),
        jxy[1] + dist_m * math.cos(theta)
    )

legs_dict = {}
for _, row in junctions_gdf.iterrows():
    j_id = row['INTER_ID']
    jxy = (row.geometry.x, row.geometry.y)
    INC_LINKS = row['INC_LINKS']

    # 1) Build raw_legs
    raw_legs = []
    for sid in INC_LINKS:
        rd = roads_dict[sid]
        geom    = rd['geometry']
        st_name = rd['STREETNAME']
        divided = (rd['StOperNEU'] == 11)

        brg = calc_bearing(jxy, geom)
        if brg is not None:
            raw_legs.append({
                'LINKS': [sid],
                'ST_NAME': st_name,
                'AVG_BRG': brg,
                'DIVIDED': divided
            })

    # 2) Sort by bearing if not empty
    df = pd.DataFrame(raw_legs)
    if not df.empty and 'AVG_BRG' in df.columns:
        df = df.sort_values('AVG_BRG', ascending=False).reset_index(drop=True)
    else:
        df = pd.DataFrame(columns=['LINKS','ST_NAME','AVG_BRG','DIVIDED'])

    # 3) Merge adjacent divided links if n>3
    combined_legs = []
    used = set()
    n = len(df)

    if n > 3:
        for i in range(n):
            if i in used:
                continue
            leg_i = df.iloc[i].to_dict()
            j = (i + 1) % n
            if j not in used and n > 1:
                leg_j = df.iloc[j].to_dict()
                if leg_i['DIVIDED'] and leg_j['DIVIDED'] and leg_i['ST_NAME'] == leg_j['ST_NAME']:
                    combined_legs.append({
                        'LINKS': leg_i['LINKS'] + leg_j['LINKS'],
                        'ST_NAME': leg_i['ST_NAME'],
                        'AVG_BRG': (leg_i['AVG_BRG'] + leg_j['AVG_BRG']) / 2.0,
                        'DIVIDED': True
                    })
                    used.update([i, j])
                    continue
            combined_legs.append(leg_i)
            used.add(i)
    else:
        combined_legs = [df.iloc[k].to_dict() for k in range(n)]

    # 4) Re-sort final & assign rank
    if combined_legs:
        final_df = pd.DataFrame(combined_legs)
        if not final_df.empty and 'AVG_BRG' in final_df.columns:
            final_df = final_df.sort_values('AVG_BRG', ascending=False).reset_index(drop=True)
            final_df['CC_RANK'] = final_df.index + 1
        else:
            final_df = pd.DataFrame(columns=['LINKS','ST_NAME','AVG_BRG','DIVIDED','CC_RANK'])
    else:
        final_df = pd.DataFrame(columns=['LINKS','ST_NAME','AVG_BRG','DIVIDED','CC_RANK'])

    legs_dict[j_id] = {
        'INTERSECTION_XY': jxy,
        'J_NODES': [j_id],
        'LEGS_DF': final_df
    }

print("legs_dict created successfully.")

legs_dict created successfully.


In [7]:
#############################
# Create Crossing Geometries
#############################

def create_crossings(legs_dictionary, roads_geo_df, out_path):
    cross_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries())
    cross_gdf['geometry'] = None
    offset = 5
    row_idx = 0

    for j_id, j_data in legs_dictionary.items():
        df = j_data['LEGS_DF']
        jxy = j_data['INTERSECTION_XY']

        if len(df) >= 3:
            n_legs = len(df)
            for i_ in range(n_legs):
                for j_ in range(n_legs):
                    if i_ == j_:
                        continue

                    leg_from = df.iloc[i_].to_dict()
                    leg_to   = df.iloc[j_].to_dict()
                    fr_rank  = leg_from.get('CC_RANK', 9999)
                    to_rank  = leg_to.get('CC_RANK', 9999)
                    num_between = ((to_rank - fr_rank) % n_legs) - 1

                    if num_between == 1:
                        x_rank = ((fr_rank + num_between - 1) % n_legs) + 1
                        leg_x  = df.loc[df['CC_RANK'] == x_rank]
                        if leg_x.empty:
                            continue
                        leg_xd = leg_x.iloc[0].to_dict()

                        fb, tb = leg_from['AVG_BRG'], leg_to['AVG_BRG']
                        if not (isinstance(fb, (int, float)) and isinstance(tb, (int, float))):
                            continue

                        sp = point_from_bearing(jxy, fb, 6)
                        ep = point_from_bearing(jxy, tb, 6)
                        line = LineString([sp, ep])

                        try:
                            line_off = line.parallel_offset(offset, 'right', join_style=2, mitre_limit=2)
                        except:
                            line_off = line

                        cross_gdf.at[row_idx, 'geometry']  = line_off
                        cross_gdf.at[row_idx, 'JUNC_ID']   = j_id
                        cross_gdf.at[row_idx, 'FRM_RANK']  = fr_rank
                        cross_gdf.at[row_idx, 'TO_RANK']   = to_rank
                        cross_gdf.at[row_idx, 'CRS_RANK']  = x_rank
                        cross_gdf.at[row_idx, 'FRM_LEG']   = str(leg_from['LINKS'])
                        cross_gdf.at[row_idx, 'TO_LEG']    = str(leg_to['LINKS'])
                        cross_gdf.at[row_idx, 'CRS_LEG']   = str(leg_xd['LINKS'])
                        cross_gdf.at[row_idx, 'FRM_STNM']  = leg_from['ST_NAME']
                        cross_gdf.at[row_idx, 'TO_STNM']   = leg_to['ST_NAME']
                        cross_gdf.at[row_idx, 'CRS_STNM']  = leg_xd['ST_NAME']
                        row_idx += 1

    cross_gdf.crs = roads_geo_df.crs
    cross_gdf.to_file(out_path)
    print(f"Saved {len(cross_gdf)} crossings to base directory.")


create_crossings(legs_dict, roads_gdf, crossings_path)

Saved 40825 crossings to base directory.


In [8]:
junctions_gdf['WAS_MERGED'].value_counts()

WAS_MERGED
False    11720
True       632
Name: count, dtype: int64